# Script para ler o Diário Oficial do Município de São Paulo

## Script BASE

### Importando as bibliotecas

In [ ]:
from datetime import date, datetime, timedelta
from PyPDF2 import PdfFileReader, PdfFileMerger
from collections import OrderedDict
import os
import fitz as fz
import tabula
import scripts

### Obter data atual

In [ ]:
def get_date_today():
    '''Obtém a data atual e retorna uma string: (YYYY-mm-dd)'''
    dt_now = date.today()
    if dt_now.weekday() == 0:
            dt_now = dt_now + timedelta(-2)
            dt_now = dt_now.strftime("%Y-%m-%d")
    else:
            dt_now = dt_now.strftime("%Y-%m-%d")
    
    return dt_now

dt_now = get_date_today()

### Criando o diretório SAVE_DIR

In [ ]:
BASE_DIR = os.getcwd()
os.chdir(BASE_DIR)
try:
    os.makedirs(f'{BASE_DIR}\\outputs_files\\{dt_now}')
    print(f'### MAKEDIRS CONCLUÍDO - Verifique a pasta: {BASE_DIR}\\outputs_files\\{dt_now}')
except:
    print(f'### MAKEDIRS PASTA EXISTENTE - Verifique a pasta: {BASE_DIR}\\outputs_files\\{dt_now}')
    pass

### Obter diretórios

In [ ]:
def get_directories():
        '''Obtém  os diretórios de trabalho.
                return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES
        '''
        dt_now = get_date_today()
        BASE_DIR = os.getcwd()
        os.chdir(BASE_DIR)
        try:
                os.makedirs(f'{BASE_DIR}\\outputs_files\\{dt_now}')
        except:
                pass

        DATA_DIR = f'{BASE_DIR}\\querido-diario\\data_collection\\data\\3550308\\{str(dt_now)}'
        SAVE_DIR = f'{BASE_DIR}\\outputs_files\\{dt_now}'
        # DATA_FILE = SAVE_DIR + '\\' + [f for f in os.listdir(SAVE_DIR) if f.endswith("tsv")][0]
        DATA_FILES = os.listdir(DATA_DIR)
        DATA_FILES.sort()
        
        print(f'###   INFO:  DIRS   ###')
        print(f'  BASE_DIR:  {BASE_DIR}')
        print(f'  DATA_DIR:  {DATA_DIR}')
        print(f'  SAVE_DIR:  {SAVE_DIR}')
        #print(f' DATA_FILE:  {DATA_FILE}')
        print(f'DATA_FILES:  {DATA_FILES}')
        
        return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILES

BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILES = get_directories()

### Obtendo Informações do DOM

In [ ]:
def get_info_dom():
        '''Obtém as informações do DOM.
                return YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM        
        '''
        dt_now = get_date_today()
        BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILES = get_directories()
        YEAR_DOM = date.today().year - 1955
        HEAD_DOM ='São Paulo, ' + str(YEAR_DOM) + ' ('

        with fz.open(DATA_DIR +'\\'+ DATA_FILES[3]) as file:
                search_edition = ''
                for page in file:
                        search_edition += page.get_text()

        search_edition = search_edition[search_edition.find(HEAD_DOM):]
        EDITION_DOM = search_edition[search_edition.find('(')+1:search_edition.find(')')]
        HEAD_DOM ='São Paulo, ' + str(YEAR_DOM) + ' (' + EDITION_DOM + ')'
        PAGS_DOM = str(len(DATA_FILES))
        DOM = f'DOM_SP-{str(YEAR_DOM)}_{str(EDITION_DOM)}_{PAGS_DOM}_{str(dt_now)}'

        print('### INFO: DOM-SP ###')
        print(f'DATA: {dt_now}')
        print(f'PAGS: {PAGS_DOM}')
        print(f'HEAD: {HEAD_DOM}')
        print(f' DOM: {DOM}')
        
        return YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM,DOM

YEAR_DOM,EDITION_DOM,PAGS_DOM,HEAD_DOM,DOM = get_info_dom()

### Renomeando arquivos

In [ ]:
contexts = ''
for file_name in DATA_FILES:
        print('Obtendo conteúdo de: ' + file_name)
        with fz.open(DATA_DIR +'\\'+ file_name) as file:
                context = ""
                for page in file:
                        context += '<pag>\n'+ page.get_text() + '</pag>\n'
                
                odd_page = "São Paulo, " + str(YEAR_DOM) + " (" + str(EDITION_DOM) + ") – "
                even_page = " – São Paulo, " + str(YEAR_DOM) + " (" + str(EDITION_DOM) + ")"

                if  context.find(odd_page) != -1:
                        num_page = context[context.find(odd_page)+21:context.find('\n',context.find(odd_page))]
                elif context.find(even_page) != -1:
                        num_page = context[context.find('\n',context.find(even_page)-5)+1:context.find(even_page)]
                elif context.find("D.O.C.; " + HEAD_DOM) != -1:
                        num_page = str(len(DATA_FILES))
                elif context.find("Ano "+ str(YEAR_DOM)) != -1:
                        num_page = '1'
                else:
                        num_page = ''

        contexts += context
        os.rename(f'{DATA_DIR}//{file_name}',f'{DATA_DIR}\\pg_{(num_page.strip()).zfill(3)}-{file_name}')

DATA_FILES = os.listdir(DATA_DIR)
DATA_FILES.sort()
print(f'### RENAME CONCLUÍDO - Verifique a pasta: {DATA_DIR} ###')

### Juntando os arquivos

In [ ]:
pdf_files = [f for f in os.listdir(DATA_DIR) if f.endswith("pdf")]
merger = PdfFileMerger()

for filename in pdf_files:
    print(f'Merge de: {filename}')
    merger.append(PdfFileReader(os.path.join(DATA_DIR, filename), "rb"))

merger.write(SAVE_DIR + "\\DOM_SP-"+ str(YEAR_DOM) + "_" + str(EDITION_DOM) + "_" + str(len(DATA_FILES)) + "_" + str(dt_now)+".pdf")
print(f'### MERGE CONCLUÍDO - Verifique a pasta: {SAVE_DIR} ###')

### Convertendo para TSV FINAL

In [ ]:
print('Convertendo arquivo para TSV')
area_text = [[42.849,28.172,1209.134,215.329],[42.849,216.314,1210.119,393.621],[42.849,395.591,1210.119,574.868],[42.849,576.838,1207.164,760.055]]
#tabula.convert_into_by_batch(SAVE_DIR,output_format="tsv",area=area_text,pages="all")
tabula.convert_into_by_batch(SAVE_DIR,output_format="tsv",area=area_text,pages="all")
print(f'### CONVERT CONCLUÍDO - Verifique a pasta: {SAVE_DIR} ###')

In [ ]:
def get_directories():
        '''Obtém  os diretórios de trabalho.
                return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES
        '''
        dt_now = get_date_today()
        BASE_DIR = os.getcwd()
        os.chdir(BASE_DIR)
        try:
                os.makedirs(f'{BASE_DIR}\\outputs_files\\{dt_now}')
        except:
                pass

        DATA_DIR = f'{BASE_DIR}\\querido-diario\\data_collection\\data\\3550308\\{str(dt_now)}'
        SAVE_DIR = f'{BASE_DIR}\\outputs_files\\{dt_now}'
        DATA_FILE = SAVE_DIR + '\\' + [f for f in os.listdir(SAVE_DIR) if f.endswith("tsv")][0]
        DATA_FILES = os.listdir(DATA_DIR)
        DATA_FILES.sort()
        
        print(f'###   INFO:  DIRS   ###')
        print(f'  BASE_DIR:  {BASE_DIR}')
        print(f'  DATA_DIR:  {DATA_DIR}')
        print(f'  SAVE_DIR:  {SAVE_DIR}')
        print(f' DATA_FILE:  {DATA_FILE}')
        print(f'DATA_FILES:  {DATA_FILES}')
        
        return BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES

BASE_DIR,DATA_DIR,SAVE_DIR,DATA_FILE,DATA_FILES = get_directories()

### Salva arquivo final

In [ ]:
def save_file(SAVE_DIR,CONTEXT,NAME_FILE,FORMAT='txt'):
        '''
                Cria um arquivo com o conteúdo e salva na pasta.
                        SAVE_DIR  : Diretório para salvar o conteúdo
                        CONTEXT   : Conteúdo a ser salvo
                        NAME_FILE : Nome do arquivo para ser salvo
                        FORMAT    : Formato do arquivo final
        '''

        with open(f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}","w",encoding='utf8') as output_file:
                output_file.write(CONTEXT)
        
        return f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}"

In [ ]:
def add_file(SAVE_DIR,CONTEXT,NAME_FILE,FORMAT='txt'):
        '''
                Cria um arquivo com o conteúdo e salva na pasta.
                        SAVE_DIR  : Diretório para salvar o conteúdo
                        CONTEXT   : Conteúdo a ser salvo
                        NAME_FILE : Nome do arquivo para ser salvo
                        FORMAT    : Formato do arquivo final
        '''

        with open(f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}","a",encoding='utf8') as output_file:
                output_file.write(CONTEXT)
        
        return f"{SAVE_DIR}\\{NAME_FILE}.{FORMAT}"

### Definindo Diretórios

### Obtem contéudo do arquivo TSV

In [ ]:
with open(DATA_FILE,'r') as file:
    conteudo = file.read()

## Scripts de Listagens

### Seção SERVIDORES

In [ ]:
conteudo_alterado = conteudo

#### Limpeza dos dados

In [ ]:
def limpa_tabulacao(conteudo_alterado):
    print('### LIMPA TABULAÇÃO ###')
    print(f'Qtd de linhas antes: {len(conteudo_alterado)}')
    conteudo_alterado = conteudo_alterado.replace('\t',' ')
    conteudo_alterado = conteudo_alterado.replace('""','')
    conteudo_alterado = conteudo_alterado.replace(' \n','\n')
    conteudo_alterado = conteudo_alterado.replace('\n ','\n')
    conteudo_alterado = conteudo_alterado.replace('    ',' ')
    conteudo_alterado = conteudo_alterado.replace('   ',' ')
    conteudo_alterado = conteudo_alterado.replace('  ',' ')
    conteudo_alterado = conteudo_alterado.replace('-\n','')
    conteudo_alterado = conteudo_alterado.replace(',\n',', ')
    conteudo_alterado = conteudo_alterado.replace(', \n',', ')
    conteudo_alterado = conteudo_alterado.replace('/\n','/')
    conteudo_alterado = conteudo_alterado.upper()
    print(f'Qtd de linhas depois: {len(conteudo_alterado)}')
    return conteudo_alterado

In [ ]:
def normaliza_cabecalho(conteudo_alterado):
    print('### NORMALIZA CABEÇALHO ###')
    print(f'Qtd de linhas antes: {len(conteudo_alterado)}')
    conteudo_alterado = conteudo_alterado.replace('E.H.','EH')
    conteudo_alterado = conteudo_alterado.replace('REG. FUNC.','RF')
    conteudo_alterado = conteudo_alterado.replace('REG.FUNC.','RF')
    conteudo_alterado = conteudo_alterado.replace('R.F.','RF')
    conteudo_alterado = conteudo_alterado.replace('E.H.','EH')
    conteudo_alterado = conteudo_alterado.replace(' VINC.','VINCULO')
    conteudo_alterado = conteudo_alterado.replace('DURAÇÃO','DURACAO')
    conteudo_alterado = conteudo_alterado.replace('A PARTIR DE','A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('A PARTIR','A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace(' ART.','ARTIGO')
    conteudo_alterado = conteudo_alterado.replace('PERÍODO','PERIODO')
    conteudo_alterado = conteudo_alterado.replace(' VÍNC.',' VINCULO')
    conteudo_alterado = conteudo_alterado.replace(' QTE DE DIAS ',' DIAS ')
    conteudo_alterado = conteudo_alterado.replace('RF. ','RF ')
    conteudo_alterado = conteudo_alterado.replace('RF NOME DIAS A PARTIR DE ARTIGO','RF NOME DIAS A_PARTIR_DE ARTIGO')
    conteudo_alterado = conteudo_alterado.replace('RF NOME A PARTIR DE','RF NOME A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF NOME DUR A PARTIR DE','RF NOME DURACAO A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF SERVIDOR CONTROLE NO','RF NOME CONTROLE_NO')
    conteudo_alterado = conteudo_alterado.replace('RF NOME CONTROLE NO','RF NOME CONTROLE_NO')
    conteudo_alterado = conteudo_alterado.replace('RF/V NOME DIAS/EXERCÍCIO A PARTIR DE','RF/VINCULO NOME DIAS A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF/VINCULO NOME DIAS A_PARTIR_DE','RF/VINCULO NOME DIAS A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF/VINC NOME NÍVEL CAT. SÍMBOLO A PARTIR DE','RF/VINCULO NOME NIVEL_CAT SIMBOLO A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF NOME CARGO','RF NOME CARGO')
    conteudo_alterado = conteudo_alterado.replace('RF V NOME EH A PARTIR DE MOTIVO','RF VINCULO NOME EH A_PARTIR_DE MOTIVO')
    conteudo_alterado = conteudo_alterado.replace('RF VINCULO NOME CARGO A PARTIR DE','RF VINCULO NOME CARGO A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('NOME RF DATA HORARIO','NOME RF DATA_HORARIO')
    conteudo_alterado = conteudo_alterado.replace('NOME RF DATAHORARIO','NOME RF DATA_HORARIO')
    conteudo_alterado = conteudo_alterado.replace('REGISTRO VINC. NOME NIIVEL CAT.','RF VINCULO NOME NIIVEL CAT')
    conteudo_alterado = conteudo_alterado.replace('REGISTRO VINC. NOME NIIVEL CAT. SIM','RF VINCULO NOME NIVEL CAT SIM')
    conteudo_alterado = conteudo_alterado.replace('RF VINC NOME NIIVEL CAT','RF VINCULO NOME NIVEL CAT')
    conteudo_alterado = conteudo_alterado.replace('REG. FUNC. NOME CARGO REGIME EXP. AC. NO','RF NOME CARGO REGIME EXP_AC_NO')
    conteudo_alterado = conteudo_alterado.replace('EH RF NOME DURAÇÃO A PARTIR ART','EH RF NOME DURACAO A_PARTIR_DE ARTIGO')
    conteudo_alterado = conteudo_alterado.replace('REG.FUNC. NOME DE PARA','RF NOME DE PARA')
    conteudo_alterado = conteudo_alterado.replace('RF NOME NOTA INDIVIDUAL NOTA INSTITUCIONAL NOTA FINAL','RF NOME NOTA_INDIVIDUAL NOTA_INSTITUCIONAL NOTA_FINAL')
    conteudo_alterado = conteudo_alterado.replace('RF NOME DURAÇÃO À PARTIR DE','RF NOME DURACAO A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF NOME PERCENTUAL BASE DE CÁLCULO DATA','RF NOME PERCENTUAL BASE_DE_CALCULO DATA')
    conteudo_alterado = conteudo_alterado.replace('REGISTRO NOME CARGO EXERCÍCIO QUAN- A PARTIR DE FUNCIONAL TIDADE','RF NOME CARGO EXERCICIO QUANTIDADE A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF VINCULO NOME: PERÍODO DE :','RF VINCULO NOME PERIODO_DE')
    conteudo_alterado = conteudo_alterado.replace('RF/V NOME CARGO N°DIAS','RF/VINCULO NOME CARGO DIAS')
    conteudo_alterado = conteudo_alterado.replace('NOME RF A PARTIR DE','NOME RF A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF/V NOME A PARTIR DE','RF/VINCULO NOME A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF: NOME: PROCESSO: E.H.:','RF NOME PROCESSO EH')
    conteudo_alterado = conteudo_alterado.replace('EH RF/V NOME DIAS A PARTIR DE','EH RF/VINCULO NOME DIAS A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF NOME E.H. A PARTIR','RF NOME E.H. A_PARTIR')
    conteudo_alterado = conteudo_alterado.replace('RF/V NOME DIAS A PARTIR DE','RF/VINCULO NOME DIAS A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('RF VINCULO NOME: PERIODO DE :','RF VINCULO NOME PERIODO')
    conteudo_alterado = conteudo_alterado.replace('RF NOME CONTROLE_NO ','RF NOME CONTROLE_NO\n')
    conteudo_alterado = conteudo_alterado.replace('RF VINCULO NOME CARGO A_PARTIR_DE ','RF VINCULO NOME CARGO A_PARTIR_DE\n')
    conteudo_alterado = conteudo_alterado.replace('RF/VINCULO NOME DIAS A_PARTIR_DE','RF/VINCULO NOME DIAS A_PARTIR_DE\n')
    conteudo_alterado = conteudo_alterado.replace('RF NOME CARGO REGIME EXP_AC_NO ','RF NOME CARGO REGIME EXP_AC_NO\n')
    conteudo_alterado = conteudo_alterado.replace('RF NOME DE PARA ','RF NOME DE PARA\n')
    conteudo_alterado = conteudo_alterado.replace('RF NOME NOTA_INDIVIDUAL NOTA_INSTITUCIONAL NOTA_FINAL ','RF NOME NOTA_INDIVIDUAL NOTA_INSTITUCIONAL NOTA_FINAL\n')
    conteudo_alterado = conteudo_alterado.replace('RF NOME DURACAO A_PARTIR_DE ','RF NOME DURACAO A_PARTIR_DE\n')
    conteudo_alterado = conteudo_alterado.replace('RF NOME PERCENTUAL BASE_DE_CALCULO DATA ','RF NOME PERCENTUAL BASE_DE_CALCULO DATA\n')
    conteudo_alterado = conteudo_alterado.replace('REGISTRO NOME CARGO EXERCÍCIO QUAN- A PARTIR DE FUNCIONAL TIDADE','RF NOME CARGO EXERCICIO QUANTIDADE A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('NOME RF A_PARTIR_DE ','NOME RF A_PARTIR_DE\n')
    conteudo_alterado = conteudo_alterado.replace('RF/V NOME A_PARTIR_DE ','RF/V NOME A_PARTIR_DE\n')
    conteudo_alterado = conteudo_alterado.replace('RF NOME A_PARTIR_DE ','RF NOME A_PARTIR_DE')
    conteudo_alterado = conteudo_alterado.replace('REGISTRO NOME CARGO EXERCÍCIO QUANTIDADE A_PARTIR_DE FUNCIONAL','RF NOME CARGO EXERCICIO QUANTIDADE A_PARTIR_DE')
    print(f'Qtd de linhas depois: {len(conteudo_alterado)}')
    return conteudo_alterado

In [ ]:
def limpa_conjuncao_preposicao(conteudo_alterado):
    print('### LIMPA CONJUNÇÃO E PREPOSIÇÃO ###')
    print(f'Qtd de linhas antes: {len(conteudo_alterado)}')
    conteudo_alterado = conteudo_alterado.replace(' AO\n',' AO ')
    conteudo_alterado = conteudo_alterado.replace('\nAO ',' AO ')
    conteudo_alterado = conteudo_alterado.replace(' AOS\n',' AOS ')
    conteudo_alterado = conteudo_alterado.replace('\nAOS ',' AOS ')
    conteudo_alterado = conteudo_alterado.replace(' SÃO\n',' SÃO ')
    conteudo_alterado = conteudo_alterado.replace(' À\n',' À ')
    conteudo_alterado = conteudo_alterado.replace('\nÀ ',' À ')
    conteudo_alterado = conteudo_alterado.replace(' ÀS\n',' ÀS ')
    conteudo_alterado = conteudo_alterado.replace('\nÀS ',' ÀS ')
    conteudo_alterado = conteudo_alterado.replace(' AS\n',' AS ')
    conteudo_alterado = conteudo_alterado.replace('\nAS ',' AS ')
    conteudo_alterado = conteudo_alterado.replace(' E\n',' E ')
    conteudo_alterado = conteudo_alterado.replace(' O\n',' O ')
    conteudo_alterado = conteudo_alterado.replace(' OS\n',' OS ')
    conteudo_alterado = conteudo_alterado.replace('\nOS ',' OS ')
    conteudo_alterado = conteudo_alterado.replace(' DA\n',' DA ')
    conteudo_alterado = conteudo_alterado.replace('\nDA ',' DA ')
    conteudo_alterado = conteudo_alterado.replace(' DAS\n',' DAS ')
    conteudo_alterado = conteudo_alterado.replace('\nDAS ',' DAS ')
    conteudo_alterado = conteudo_alterado.replace(' DA(S)\n',' DA(S) ')
    conteudo_alterado = conteudo_alterado.replace('\nDA(S) ',' DA(S) ')
    conteudo_alterado = conteudo_alterado.replace(' DE\n',' DE ')
    conteudo_alterado = conteudo_alterado.replace('\nDE ',' DE ')
    conteudo_alterado = conteudo_alterado.replace(' DO\n',' DO ')
    conteudo_alterado = conteudo_alterado.replace('\nDO ',' DO ')
    conteudo_alterado = conteudo_alterado.replace(' DOS\n',' DOS ')
    conteudo_alterado = conteudo_alterado.replace('\nDOS ',' DOS ')
    conteudo_alterado = conteudo_alterado.replace(' DO(S)\n',' DO(S) ')
    conteudo_alterado = conteudo_alterado.replace('\nDO(S) ',' DO(S) ')
    conteudo_alterado = conteudo_alterado.replace(' COM\n',' COM ')
    conteudo_alterado = conteudo_alterado.replace('\nCOM ',' COM ')
    conteudo_alterado = conteudo_alterado.replace(' EM\n',' EM ')
    conteudo_alterado = conteudo_alterado.replace('\nEM ',' EM ')
    conteudo_alterado = conteudo_alterado.replace(' PARA\n',' PARA ')
    conteudo_alterado = conteudo_alterado.replace('\nPARA ',' PARA ')
    conteudo_alterado = conteudo_alterado.replace(' P/ ',' PARA ')
    conteudo_alterado = conteudo_alterado.replace(' POR\n',' POR ')
    conteudo_alterado = conteudo_alterado.replace(' NA\n',' NA ')
    conteudo_alterado = conteudo_alterado.replace(' NAS\n',' NAS ')
    conteudo_alterado = conteudo_alterado.replace('\nNAS ',' NAS ')
    conteudo_alterado = conteudo_alterado.replace(' NO\n',' NO ')
    conteudo_alterado = conteudo_alterado.replace(' NOS\n',' NOS ')
    conteudo_alterado = conteudo_alterado.replace('\nNOS ',' NOS ')
    print(f'Qtd de linhas depois: {len(conteudo_alterado)}')
    return conteudo_alterado

In [ ]:
conteudo_alterado = limpa_tabulacao(conteudo_alterado)
conteudo_alterado = limpa_conjuncao_preposicao(conteudo_alterado)
conteudo_alterado = limpa_tabulacao(conteudo_alterado)
conteudo_alterado = normaliza_cabecalho(conteudo_alterado)
conteudo_alterado = limpa_tabulacao(conteudo_alterado)
conteudo_alterado = limpa_conjuncao_preposicao(conteudo_alterado)
conteudo_alterado = limpa_tabulacao(conteudo_alterado)

In [ ]:
print(f'Qtd de linhas antes: {len(conteudo_alterado)}')
conteudo_alterado = conteudo_alterado.replace('FUNDAÇÃO PAULISTANA DE EDUCAÇÃO\nTECNOLOGIA E CULTURA','FUNDAÇÃO PAULISTANA DE EDUCAÇÃO TECNOLOGIA E CULTURA')
conteudo_alterado = conteudo_alterado.replace('HOSPITAL DO SERVIDOR\nPÚBLICO MUNICIPAL','HOSPITAL DO SERVIDOR PÚBLICO MUNICIPAL')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE ASSIST E DESENV\nSOCIAL','SECRETARIA MUNICIPAL DE ASSIST E DESENV SOCIAL')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE INFRAESTRUTURA\nURBANA E OB','SECRETARIA MUNICIPAL DE INFRAESTRUTURA URBANA E OB')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DE SEGURANCA\nURBANA','SECRETARIA MUNICIPAL DE SEGURANCA URBANA')
conteudo_alterado = conteudo_alterado.replace('SECRETARIA MUNICIPAL DO VERDE E DO MEIO\nAMBIENTE','SECRETARIA MUNICIPAL DO VERDE E DO MEIO AMBIENTE')
conteudo_alterado = conteudo_alterado.replace('COORDENADORIA DE ADMINISTRAÇÃO\nE FINANÇAS','COORDENADORIA DE ADMINISTRAÇÃO E FINANÇAS')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS\nEM GRAU DE RECURSO','LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS\nNOS TERMOS DA LEI 8989/79','RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA AVALIAÇÃO DE ESPECIALISTA','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE FALTAS DOS CONVOCADOS\nPARA JUNTA MÉDICA PARA FINS DE PENSÃO\nPOR MORTE','RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM\nFUNÇÃO DE:','RELAÇÃO DE ALTA DE AT/DT RETIFICADA EM FUNÇÃO DE:')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE SERVIDORES SUBMETIDOS À\nAVALIAÇÃO DE ESPECIALISTA\nCOM CAPACIDADE LABORATIVA PARA A SUA\nFUNÇÃO ORIGINAL','RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE ESPECIALISTA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO ORIGINAL')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO\nDEFERIDO','RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS DE CURTA DURAÇÃO\nDOC ','LICENÇAS MÉDICAS DE CURTA DURAÇÃO DOC ')
conteudo_alterado = conteudo_alterado.replace('ACI-\nDENTE','ACIDENTE')
conteudo_alterado = conteudo_alterado.replace('CANCELA-\nDAS','CANCELADAS')
conteudo_alterado = conteudo_alterado.replace('RETIFICA-\nDAS','RETIFICADAS')
conteudo_alterado = conteudo_alterado.replace('RECO-\nMENDAÇÃO','RECOMENDAÇÃO')
conteudo_alterado = conteudo_alterado.replace('RECOMENDAÇÃO\nD','RECOMENDAÇÃO D')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS\nFACE','LICENÇAS MÉDICAS FACE')
conteudo_alterado = conteudo_alterado.replace('LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA\nPREVIDÊNCIA SOCIAL\nCONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO\n','LICENÇAS MÉDICAS PARA SERVIDORES SOB REGIME GERAL DA PREVIDÊNCIA SOCIAL CONCEDIDA DE ACORDO COM O DETERMINADO NA PORTARIA NO ')
conteudo_alterado = conteudo_alterado.replace('LICENÇA\nMÉDICA','LICENÇA MÉDICA')
conteudo_alterado = conteudo_alterado.replace('DECRETO NO\n','DECRETO NO')
conteudo_alterado = conteudo_alterado.replace('PUBLICADO\nNO','PUBLICADO NO')
conteudo_alterado = conteudo_alterado.replace('\nCONCEDIDA',' CONCEDIDA')
conteudo_alterado = conteudo_alterado.replace('\nCONCEDIDAS ',' CONCEDIDAS ')
conteudo_alterado = conteudo_alterado.replace(' RETIFICA- DAS ',' RETIFICADAS ')
conteudo_alterado = conteudo_alterado.replace('PORTARIA\n','PORTARIA ')
conteudo_alterado = conteudo_alterado.replace(' TRABALHO\nINDEFERIDO',' TRABALHO INDEFERIDO')
conteudo_alterado = conteudo_alterado.replace(' DOC\nSEI ',' DOC SEI ')
conteudo_alterado = conteudo_alterado.replace(' DA LEI\n',' DA LEI ')
conteudo_alterado = conteudo_alterado.replace(' NOMES DOS SERVIDORES ','\nNOMES DOS SERVIDORES ')
conteudo_alterado = conteudo_alterado.replace(' RELAÇÃO DOS','\nRELAÇÃO DOS')
conteudo_alterado = conteudo_alterado.replace('RELAÇÃO DOS ESTAGIÁRIOS CONTRATADOS E DESLIGADOS,','RELAÇÃO DOS ESTAGIÁRIOS CONTRATADOS E DESLIGADOS\n')
print(f'Qtd de linhas depois: {len(conteudo_alterado)}')


In [ ]:
# Localiza o início das seções SERVIDORES e CONCURSOS
sc_servidores = conteudo_alterado.find('\nSERVIDORES\n')
sc_concursos = conteudo_alterado.find('\nCONCURSOS\n')
print(f"SERVIRODRES: {sc_servidores} / CONCURSOS: {sc_concursos}")
conteudo_servidores = conteudo_alterado[sc_servidores:sc_concursos]
#conteudo_servidores = conteudo

In [ ]:
#print(conteudo_servidores)
save_file(SAVE_DIR,conteudo_servidores,'00-SECAO_SERVIDORES')

In [ ]:
#print(conteudo_servidores)

### Relação de Licença Médica

#### CONTEÚDO PRINCIPAL

In [ ]:
conteudo_licenca_medica = conteudo_servidores

In [ ]:
licencas_mapeadas = [
    'RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO',
    'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO',
    'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO EM GRAU DE RECURSO',
    'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO INDEFERIDO',
    'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA',
    'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA',
    'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA',
    'RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA',
    'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE',
    'RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79',
    'RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79',
    'RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO',
    'ENGLOBAMENTO DE LICENÇAS MÉDICAS FACE PORTARIA 11/84',
    'LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO',
]

In [ ]:
rl_lm =  conteudo_licenca_medica.split('\n')
licencas = OrderedDict({})
str_listagens = 'ID;LISTAGEM;INICIO;MAPEADA;QTD\n'
linha = 0
lin_anterior = 0
id = 0
listas = []

for line in rl_lm:
    mapeada = line in licencas_mapeadas
    if ('LICENÇAS MÉDICAS' in line or 'LICENÇA NOJO' in line) and not 'SEÇÃO DE LICENÇAS MÉDICAS' in line:
        id += 1
        licencas.update({
            line : linha
        })
        listas.append(linha)
        if id == 1:
            str_listagens += f'{id};{line};{linha};{mapeada};'
            lin_anterior = linha
        if id > 1:
            qtd = linha - lin_anterior
            str_listagens += f'{qtd}\n{id};{line};{linha};{mapeada};'
            lin_anterior = lin_anterior
        #print(line)
    elif ('RELAÇÃO DE' in line or 'RELAÇÃO DOS' in line) or 'RELAÇÃO DE LICENÇA MÉDICA' in line:
        id += 1
        licencas.update({
            line : linha
        })
        listas.append(linha)
        if id == 1:
            str_listagens += f'{id};{line};{linha};{mapeada};'
            lin_anterior = linha
        if id > 1:
            qtd = linha - lin_anterior
            str_listagens += f'{qtd}\n{id};{line};{linha};{mapeada};'
            lin_anterior = lin_anterior
        #print(line)
    elif 'SERVIDORES COM ' in line or 'NOMES DOS SERVIDORES' in line:
        id += 1
        licencas.update({
            line : linha
        })
        if id == 1:
            str_listagens += f'{id};{line};{linha};{mapeada};'
            lin_anterior = linha
        if id > 1:
            qtd = linha - lin_anterior
            str_listagens += f'{qtd}\n{id};{line};{linha};{mapeada};'
            lin_anterior = lin_anterior
        #print(line)
    elif 'HORÁRIO AMAMENTAÇÃO – DEFERIDO' in line or 'FÉRIAS DEFERIDAS' in line:
        id += 1
        licencas.update({
            line : linha
        })
        if id == 1:
            str_listagens += f'{id};{line};{linha};{mapeada};'
            lin_anterior = linha
        if id > 1:
            qtd = linha - lin_anterior
            str_listagens += f'{qtd}\n{id};{line};{linha};{mapeada};'
            lin_anterior = lin_anterior
        #print(line)

    #if 'RF' in line or 'REG. FUN.' in line or 'REG.FUN.' in line:
        #print(line)
    linha += 1

#print(licencas)
#print(OrderedDict(licencas))
#print(listas)
print(f'\nTotal de listas: {len(listas)}')
print(str_listagens)

In [ ]:
id = 0
str_licencas = 'ID;LISTAGEM;INICIO;MAPEADA;QTD\n'

for relacao, linha in licencas.items():
    id += 1
    mapeada = relacao in licencas_mapeadas
    if id == 1:
        str_licencas += f'{id};{relacao};{linha};{mapeada};'
        lin_anterior = linha
    if id > 1:
        qtd = linha - lin_anterior
        str_licencas += f'{qtd}\n{id};{relacao};{linha};{mapeada};'
        lin_anterior = lin_anterior

#print(str_licencas)
save_file(SAVE_DIR,str_licencas,'00-LISTAGENS_DOM','csv')

#### RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79

In [ ]:
RELACAO = 'RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

In [ ]:
'''Listagem: RELAÇÃO DE LICENÇAS MÉDICAS NOS TERMOS DA LEI 8989/79
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[2:-3]
        4. DIAS         : numeric()  s_item[-3]
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    #print(f'analise: {s_item}')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5:
        if len(s_item) >= 5 and s_item[0][-1].isnumeric() and s_item[-3][0].isnumeric() and s_item[-2][0].isnumeric():
            rf = s_item[0].split('/')[0]
            vinc = s_item[0].split('/')[1]
            nome = str(' ').join(s_item[1:-3])
            str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
            #print(str_list_items)
    elif len(s_item) >= 2 and s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        print(str_list_items)
        VALIDA_LINHA = False
    i += 1
    #print(i)

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79

In [ ]:
RELACAO = 'RELAÇÃO DE LICENÇAS MÉDICAS NEGADAS NOS TERMOS DA LEI 8989/79'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items


In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-2]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][-1].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE

In [ ]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA JUNTA MÉDICA PARA FINS DE PENSÃO POR MORTE'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA

In [ ]:
RELACAO = 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [ ]:
i = 0
SECRETARIA = ''
str_list_items = 'RF;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric():
        if s_item[-3].isnumeric():
            nome = str(' ').join(s_item[1:-3])
            str_list_items += f'{s_item[0]};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO}/{DOM}\n'
            #print(str_list_items)
        else:
            nome = str(' ').join(s_item[1:-2])
            str_list_items += f'{s_item[0]};{nome};;{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
            #print(str_list_items)
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO

In [ ]:
RELACAO = 'LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

In [ ]:
'''Listagem: LICENÇAS MÉDICAS CONCEDIDAS/NEGADAS EM GRAU DE RECURSO
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[2:-3]
        4. DIAS         : numeric()  s_item[-3]
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][-1].isnumeric() and s_item[-3][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA

In [ ]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS PARA AVALIAÇÃO DE ESPECIALISTA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. A_PARTIR_DE  : date()     s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DATA;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA

In [ ]:
RELACAO = 'RELAÇÃO DE FALTAS DOS CONVOCADOS P/ AVALIAÇÃO DE ISENÇÃO DE IMPOSTO DE RENDA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. A_PARTIR_DE  : date()     s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;A_PARITR_DE;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA

In [ ]:
RELACAO = 'RELAÇÃO DE FALTA DO SERVIDOR À PERÍCIA'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-2]
        4. A_PARTIR_DE  : date()     s_item[-2]
        5. ARTIGO       : str()      s_item[-1]
'''
    
i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-2])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO

In [ ]:
RELACAO = 'RELAÇÃO DE SERVIDORES SUBMETIDOS À AVALIAÇÃO DE CAPACIDADE LABORATIVA COM CAPACIDADE LABORATIVA PARA A SUA FUNÇÃO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:])
        str_list_items += f'{rf};{vinc};{nome};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:])
        str_list_items += f'{rf};{vinc};{nome};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,RELACAO,'csv')

#### ENGLOBAMENTO DE LICENÇAS MÉDICAS FACE PORTARIA 11/84

In [ ]:
RELACAO = 'ENGLOBAMENTO DE LICENÇAS MÉDICAS FACE PORTARIA 11/84'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-3]
        4. DIAS         : numeric()  s_item[-3]
        5. A_PARTIR_DE  : date()     s_item[-2]
        6. ARTIGO       : str()      s_item[-1]
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-3][0].isnumeric() and s_item[-2][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-3])
        str_list_items += f'{rf};{vinc};{nome};{s_item[-3]};{s_item[-2]};{s_item[-1]};{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
COMPLEMENTO = str('_').join('RELAÇÃO DE LICENÇAS MÉDICAS RETIFICADAS EM FUNÇÃO DE:'.replace(':','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

#### RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO

In [ ]:
RELACAO = 'RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
#list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME A_PARTIR_DE' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

#### RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO :

In [ ]:
RELACAO = 'RELAÇÃO DE ALTAS DO ACIDENTE DO TRABALHO E DA DOENÇA DO TRABALHO :'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''
LISTAS = [
            'APÓS TRANSFORMAÇÃO DO ARTIGO DA LICENÇA MÉDICA',
        ]

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if item in LISTAS:
            RELACAO2 = f'{RELACAO} {item}'
    if 'RF NOME A_PARTIR_DE' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').replace(':','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

#### RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO

In [ ]:
RELACAO = 'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME DIAS A_PARTIR_DE ARTIGO' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 5 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

#### RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO EM GRAU DE RECURSO

In [ ]:
RELACAO = 'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO DEFERIDO EM GRAU DE RECURSO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
else:
    list_items = []
list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    s_item = item.strip().split(' ')
    if 'RF NOME A_PARTIR_DE' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')

#### RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO INDEFERIDO

In [ ]:
RELACAO = 'RELAÇÃO DE CONCESSÃO DE NEXO DE ACIDENTE DO TRABALHO INDEFERIDO'

if RELACAO in licencas:
    list_items = rl_lm[licencas[RELACAO]:listas[listas.index(licencas[RELACAO])+1]]
    #print(licencas[RELACAO])
    #print(listas[listas.index(licencas[RELACAO])+1])
else:
    list_items = []
list_items

In [ ]:
'''
    COLUNAS:
        1. RF           : numeric(7) s_item[0][0]
        2. VINC         : numeric(1) s_item[0][1]
        3. NOME         : str()      s_item[1:-1]
        4. DIAS         : null
        5. A_PARTIR_DE  : date()     s_item[-1]
        6. ARTIGO       : null
'''

i = 0
SECRETARIA = ''
VALIDA_LINHA = ''
str_list_items = 'RF;VINC;NOME;DIAS;A_PARTIR_DE;ARTIGO;SECRETARIA;LISTA;DOM\n'

for item in list_items:
    print(item)
    s_item = item.strip().split(' ')
    if 'RF NOME A_PARTIR_DE' in item:
        SECRETARIA = list_items[i-1]
    elif len(s_item) >= 4 and s_item[0][0].isnumeric() and s_item[-1][0].isnumeric():
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
    elif s_item[0][-1].isnumeric() and VALIDA_LINHA == False:
        linha = s_item
        VALIDA_LINHA = True
    elif VALIDA_LINHA == True:
        linha += f' {item}'
        s_item = item.split(' ')
        rf = s_item[0].split('/')[0]
        vinc = s_item[0].split('/')[1]
        nome = str(' ').join(s_item[1:-1])
        str_list_items += f'{rf};{vinc};{nome};;{s_item[-1]};;{SECRETARIA};{RELACAO};{DOM}\n'
        #print(str_list_items)
        VALIDA_LINHA = False
    i += 1
    print(i)

In [ ]:
RELACAO = str('_').join(RELACAO.replace('/','_').split(' '))
save_file(SAVE_DIR,str_list_items,f'{COMPLEMENTO}{RELACAO}','csv')